## 问题定义

本问题为求解固液体系中的传热。其大致定义参考下图：

![problem](./resource/problem.png)


其中中间的小物体为固体，外围为低温流体。

具体参数如下：

### 流体参数
* 流体运动粘度：$0.004195088 m^2/s$
* 流体密度：$1.1614 kg/m^3$
* 流体比热：$1005 J/(kg \cdot K)$
* 流体热导率：$0.0261 W / (m \cdot K)$

### 固体参数
* 固体密度：$8930 kg/m^3$
* 固体比热：$385 J/(kg \cdot K)$
* 固体热导率：$385 W / (m \cdot K)$

### 环境参数
* 入口流体速度：$5.24386 m/s$
* 入口温度：$25 ℃$
* 固体热源热流：$51.948051948 W / m^2$

### 无量纲因子
* 长度尺度 (length_scale)：$0.04 m$
* 时间尺度 (time_scale)：$0.007627968710072352 s$
* 质量尺度 (mass_scale)：$7.43296e-05 kg$
* 温度尺度 (temp_scale)：$1 K$
* 速度尺度 (velocity_scale)：length_scale / time_scale $m/s$
* 压力尺度 (pressure_scale)：mass_scale / (length_scale * time_scale^2) $ kg / (m \cdot s^2)$
* 密度尺度 (density_scale)：mass_scale / length_scale^3 $kg/m^3$
* 功率尺度 (watt_scale)：(mass_scale * length_scale^2) / (time_scale^3) $kg m^2 / s^3$
* 力尺度（joule_scale）：(mass_scale * length_scale^2) / (time_scale^2) $kg m^2 / s^2$

### 几何
* channel:
    * 长：-2.5, 5.0
    * 宽：-0.5, 0.5
* chip:
    * 长：-1.0, 0.0
    * 宽：-0.5, 0.1


### 求解目标

要求求解上述求解区域的稳态解

## 基本描述

在本例中流动和传热是单向耦合。具体而言，流动影响传热，但是传热不影响流动。因此分两步求解，首先求解流动，然后将流动的模型固定并求解传热。

本文件为求解flow的代码，求解heat transfer请参考17_2D_ChipSolidFluid_HeatTransfer


## 求解flow

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [2]:
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra.utils import compose

In [3]:
import numpy as np
from sympy import Symbol, Eq, And, Or

import modulus.sym
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.utils.io import csv_to_dict
from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.geometry.primitives_2d import Rectangle, Line, Channel2D
from modulus.sym.utils.sympy.functions import parabola
from modulus.sym.eq.pdes.navier_stokes import NavierStokes
from modulus.sym.eq.pdes.basic import NormalDotVec
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
    IntegralBoundaryConstraint,
)

from modulus.sym.domain.validator import PointwiseValidator
from modulus.sym.utils.io.plotter import ValidatorPlotter, InferencerPlotter
from modulus.sym.key import Key
from modulus.sym.node import Node
from modulus.sym.models.fourier_net import FourierNetArch

In [4]:
cfg = compose(config_path="conf", config_name="config_flow")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 40000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: ${training.rec_results_freq}
  rec_inference_freq: ${training.rec_results_freq}
  rec_monitor_freq: ${training.rec_results_freq}
  rec_constraint_freq: 10000
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: true
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: false
run_mode: train
arch

### 定义必要组件

#### Geo

In [5]:
#############
# Real Params
#############
fluid_kinematic_viscosity = 0.004195088  # m**2/s
fluid_density = 1.1614  # kg/m**3
fluid_specific_heat = 1005  # J/(kg K)
fluid_conductivity = 0.0261  # W/(m K)

# copper params
copper_density = 8930  # kg/m3
copper_specific_heat = 385  # J/(kg K)
copper_conductivity = 385  # W/(m K)

# boundary params
inlet_velocity = 5.24386  # m/s
inlet_temp = 25.0  # C
copper_heat_flux = 51.948051948  # W / m2

################
# Non dim params
################
length_scale = 0.04  # m
time_scale = 0.007627968710072352  # s
mass_scale = 7.43296e-05  # kg
temp_scale = 1.0  # K
velocity_scale = length_scale / time_scale  # m/s
pressure_scale = mass_scale / (length_scale * time_scale**2)  # kg / (m s**2)
density_scale = mass_scale / length_scale**3  # kg/m3
watt_scale = (mass_scale * length_scale**2) / (time_scale**3)  # kg m**2 / s**3
joule_scale = (mass_scale * length_scale**2) / (
    time_scale**2
)  # kg * m**2 / s**2

##############################
# Nondimensionalization Params
##############################
# fluid params
nd_fluid_kinematic_viscosity = fluid_kinematic_viscosity / (
    length_scale**2 / time_scale
)
nd_fluid_density = fluid_density / density_scale
nd_fluid_specific_heat = fluid_specific_heat / (
    joule_scale / (mass_scale * temp_scale)
)
nd_fluid_conductivity = fluid_conductivity / (
    watt_scale / (length_scale * temp_scale)
)
nd_fluid_diffusivity = nd_fluid_conductivity / (
    nd_fluid_specific_heat * nd_fluid_density
)

# copper params
nd_copper_density = copper_density / (mass_scale / length_scale**3)
nd_copper_specific_heat = copper_specific_heat / (
    joule_scale / (mass_scale * temp_scale)
)
nd_copper_conductivity = copper_conductivity / (
    watt_scale / (length_scale * temp_scale)
)
nd_copper_diffusivity = nd_copper_conductivity / (
    nd_copper_specific_heat * nd_copper_density
)

# boundary params
nd_inlet_velocity = inlet_velocity / (length_scale / time_scale)
nd_inlet_temp = inlet_temp / temp_scale
nd_copper_source_grad = copper_heat_flux * length_scale / temp_scale

In [6]:
# add constraints to solver
# simulation params
channel_length = (-2.5, 5.0)
channel_width = (-0.5, 0.5)
chip_pos = -1.0
chip_height = 0.6
chip_width = 1.0

# define sympy variables to parametrize domain curves
x, y = Symbol("x"), Symbol("y")

# define geometry
channel = Channel2D(
    (channel_length[0], channel_width[0]), (channel_length[1], channel_width[1])
)  # channel
inlet = Line(
    (channel_length[0], channel_width[0]),
    (channel_length[0], channel_width[1]),
    normal=1,
)  # 入口
outlet = Line(
    (channel_length[1], channel_width[0]),
    (channel_length[1], channel_width[1]),
    normal=1,
)  # 出口
rec = Rectangle(
    (chip_pos, channel_width[0]),
    (chip_pos + chip_width, channel_width[0] + chip_height),
)  # chip
geo = channel - rec  # 流体区域
x_pos = Symbol("x_pos")
integral_line = Line((x_pos, channel_width[0]), (x_pos, channel_width[1]), 1)
x_pos_range = {
    x_pos: lambda batch_size: np.full(
        (batch_size, 1), np.random.uniform(channel_length[0], channel_length[1])
    )
}

#### PDE

In [7]:
ns = NavierStokes(
    nu=nd_fluid_kinematic_viscosity, rho=nd_fluid_density, dim=2, time=False
)  # NS方程
normal_dot_vel = NormalDotVec(["u", "v"])  # 法向速度

#### Model

In [8]:
# 定义为FourierNet
# 输入为空间坐标
# 输出为预测的速度和压强
flow_net = FourierNetArch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u"), Key("v"), Key("p")],
    frequencies=("axis", [i / 5.0 for i in range(25)]),
)

#### Node

In [9]:
nodes = (
    ns.make_nodes()  # NS方程
    + normal_dot_vel.make_nodes()  # 法向速度
    + [flow_net.make_node(name="flow_network")]  # 模型
)

#### Domain

In [10]:
domain = Domain()

#### 入口条件

In [11]:
inlet = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=inlet,
    outvar={"u": nd_inlet_velocity, "v": 0},
    batch_size=cfg.batch_size.inlet,
)
domain.add_constraint(inlet, "inlet")  # 入口速度约束

#### 出口条件

In [12]:
outlet = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=outlet,
    outvar={"p": 0},
    batch_size=cfg.batch_size.outlet,
    criteria=Eq(x, channel_length[1]),
)
domain.add_constraint(outlet, "outlet")  # 出口压强约束

#### 无滑移边界

In [13]:
no_slip = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"u": 0, "v": 0},
    batch_size=cfg.batch_size.no_slip,
)
domain.add_constraint(no_slip, "no_slip")

#### NS方程 （远离chip的区域）

In [14]:
interior_lr = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"continuity": 0, "momentum_x": 0, "momentum_y": 0},
    batch_size=cfg.batch_size.interior_lr,
    criteria=Or(x < (chip_pos - 0.25), x > (chip_pos + chip_width + 0.25)),
    lambda_weighting={
        "continuity": 2 * Symbol("sdf"),
        "momentum_x": 2 * Symbol("sdf"),
        "momentum_y": 2 * Symbol("sdf"),
    },
)
domain.add_constraint(interior_lr, "interior_lr")

#### NS方程 （靠近chip的区域）

In [15]:
interior_hr = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"continuity": 0, "momentum_x": 0, "momentum_y": 0},
    batch_size=cfg.batch_size.interior_hr,
    criteria=And(x > (chip_pos - 0.25), x < (chip_pos + chip_width + 0.25)),
    lambda_weighting={
        "continuity": 2 * Symbol("sdf"),
        "momentum_x": 2 * Symbol("sdf"),
        "momentum_y": 2 * Symbol("sdf"),
    },
)
domain.add_constraint(interior_hr, "interior_hr")

#### 积分约束

In [16]:
def integral_criteria(invar, params):
    sdf = geo.sdf(invar, params)
    return np.greater(sdf["sdf"], 0)  # 确保没有切割到chip范围内

integral_continuity = IntegralBoundaryConstraint(
    nodes=nodes,
    geometry=integral_line,
    outvar={"normal_dot_vel": 1},
    batch_size=cfg.batch_size.num_integral_continuity,
    integral_batch_size=cfg.batch_size.integral_continuity,
    lambda_weighting={"normal_dot_vel": 1},
    criteria=integral_criteria,
    parameterization=x_pos_range,
)
domain.add_constraint(integral_continuity, "integral_continuity")

#### 验证器以及其他必要组件

In [17]:
file_path = "openfoam/2d_real_cht_fluid.csv"
if os.path.exists(to_absolute_path(file_path)):
    mapping = {
        "x-coordinate": "x",
        "y-coordinate": "y",
        "x-velocity": "u",
        "y-velocity": "v",
        "pressure": "p",
        "temperature": "theta_f",
    }
    openfoam_var = csv_to_dict(to_absolute_path(file_path), mapping)
    openfoam_var["x"] = openfoam_var["x"] / length_scale - 2.5  # normalize pos
    openfoam_var["y"] = openfoam_var["y"] / length_scale - 0.5
    openfoam_var["p"] = (openfoam_var["p"] + 400.0) / pressure_scale
    openfoam_var["u"] = openfoam_var["u"] / velocity_scale
    openfoam_var["v"] = openfoam_var["v"] / velocity_scale

    openfoam_invar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["x", "y"]
    }
    openfoam_outvar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["u", "v", "p"]
    }
    openfoam_validator = PointwiseValidator(
        nodes=nodes,
        invar=openfoam_invar_numpy,
        true_outvar=openfoam_outvar_numpy,
        plotter=ValidatorPlotter(),
    )
    domain.add_validator(openfoam_validator)
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

### 求解器以及求解

In [18]:
# 定义求解器
slv = Solver(cfg, domain)

手动加载日志系统

In [19]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [20]:
slv.solve()

2024-03-01 06:54:47,121 - WARNING - Installed PyTorch version 2.2.0a0+81ea7a4 is not TorchScript supported in Modulus. Version 2.1.0a0+4136153 is officially supported.
2024-03-01 06:54:47,128 - INFO - attempting to restore from: /workspace/16_2D_ChipSolidFluid_HeatTransfer/outputs
2024-03-01 06:54:47,129 - WARNING - optimizer checkpoint not found
2024-03-01 06:54:47,130 - WARNING - model flow_network.0.pth not found
2024-03-01 06:54:48,342 - INFO - [step:          0] record constraint batch time:  1.196e-01s
2024-03-01 06:54:57,855 - INFO - [step:          0] record validators time:  9.510e+00s
2024-03-01 06:54:57,924 - INFO - [step:          0] saved checkpoint to /workspace/16_2D_ChipSolidFluid_HeatTransfer/outputs
2024-03-01 06:54:57,927 - INFO - [step:          0] loss:  6.042e+00
2024-03-01 06:55:01,126 - INFO - Attempting cuda graph building, this may take a bit...
2024-03-01 06:55:14,280 - INFO - [step:        100] loss:  2.263e+00, time/iteration:  1.635e+02 ms
2024-03-01 06:55

2024-03-01 06:59:46,392 - INFO - [step:       6200] loss:  6.353e-02, time/iteration:  2.510e+01 ms
2024-03-01 06:59:48,946 - INFO - [step:       6300] loss:  5.652e-02, time/iteration:  2.552e+01 ms
2024-03-01 06:59:51,464 - INFO - [step:       6400] loss:  4.617e-02, time/iteration:  2.517e+01 ms
2024-03-01 06:59:54,830 - INFO - [step:       6500] loss:  4.294e-02, time/iteration:  3.364e+01 ms
2024-03-01 06:59:58,490 - INFO - [step:       6600] loss:  3.681e-02, time/iteration:  3.659e+01 ms
2024-03-01 07:00:02,146 - INFO - [step:       6700] loss:  1.456e-01, time/iteration:  3.655e+01 ms
2024-03-01 07:00:05,800 - INFO - [step:       6800] loss:  2.934e-02, time/iteration:  3.652e+01 ms
2024-03-01 07:00:09,448 - INFO - [step:       6900] loss:  4.301e-02, time/iteration:  3.646e+01 ms
2024-03-01 07:00:23,303 - INFO - [step:       7000] record validators time:  9.516e+00s
2024-03-01 07:00:23,345 - INFO - [step:       7000] saved checkpoint to /workspace/16_2D_ChipSolidFluid_HeatTran

2024-03-01 07:04:54,384 - INFO - [step:      13000] saved checkpoint to /workspace/16_2D_ChipSolidFluid_HeatTransfer/outputs
2024-03-01 07:04:54,387 - INFO - [step:      13000] loss:  1.263e-02, time/iteration:  1.323e+02 ms
2024-03-01 07:04:57,492 - INFO - [step:      13100] loss:  4.204e-02, time/iteration:  3.104e+01 ms
2024-03-01 07:05:00,696 - INFO - [step:      13200] loss:  2.284e-02, time/iteration:  3.202e+01 ms
2024-03-01 07:05:04,366 - INFO - [step:      13300] loss:  2.312e-02, time/iteration:  3.668e+01 ms
2024-03-01 07:05:08,026 - INFO - [step:      13400] loss:  1.834e-02, time/iteration:  3.659e+01 ms
2024-03-01 07:05:11,697 - INFO - [step:      13500] loss:  1.631e-02, time/iteration:  3.669e+01 ms
2024-03-01 07:05:15,356 - INFO - [step:      13600] loss:  1.928e-02, time/iteration:  3.658e+01 ms
2024-03-01 07:05:19,001 - INFO - [step:      13700] loss:  1.377e-02, time/iteration:  3.643e+01 ms
2024-03-01 07:05:22,682 - INFO - [step:      13800] loss:  2.983e-02, time/

2024-03-01 07:09:55,413 - INFO - [step:      19800] loss:  7.211e-03, time/iteration:  3.644e+01 ms
2024-03-01 07:09:59,080 - INFO - [step:      19900] loss:  9.078e-03, time/iteration:  3.665e+01 ms
2024-03-01 07:10:03,071 - INFO - [step:      20000] record constraint batch time:  1.126e-01s
2024-03-01 07:10:11,927 - INFO - [step:      20000] record validators time:  8.855e+00s
2024-03-01 07:10:11,957 - INFO - [step:      20000] saved checkpoint to /workspace/16_2D_ChipSolidFluid_HeatTransfer/outputs
2024-03-01 07:10:11,959 - INFO - [step:      20000] loss:  9.721e-03, time/iteration:  1.288e+02 ms
2024-03-01 07:10:15,611 - INFO - [step:      20100] loss:  1.258e-02, time/iteration:  3.650e+01 ms
2024-03-01 07:10:19,258 - INFO - [step:      20200] loss:  2.043e-02, time/iteration:  3.645e+01 ms
2024-03-01 07:10:22,977 - INFO - [step:      20300] loss:  1.183e-02, time/iteration:  3.717e+01 ms
2024-03-01 07:10:26,644 - INFO - [step:      20400] loss:  7.665e-03, time/iteration:  3.665e

2024-03-01 07:15:03,772 - INFO - [step:      26500] loss:  9.828e-03, time/iteration:  3.645e+01 ms
2024-03-01 07:15:07,399 - INFO - [step:      26600] loss:  3.598e-03, time/iteration:  3.625e+01 ms
2024-03-01 07:15:10,481 - INFO - [step:      26700] loss:  8.455e-03, time/iteration:  3.080e+01 ms
2024-03-01 07:15:13,747 - INFO - [step:      26800] loss:  1.673e-02, time/iteration:  3.264e+01 ms
2024-03-01 07:15:17,379 - INFO - [step:      26900] loss:  5.644e-03, time/iteration:  3.629e+01 ms
2024-03-01 07:15:30,698 - INFO - [step:      27000] record validators time:  9.121e+00s
2024-03-01 07:15:30,732 - INFO - [step:      27000] saved checkpoint to /workspace/16_2D_ChipSolidFluid_HeatTransfer/outputs
2024-03-01 07:15:30,734 - INFO - [step:      27000] loss:  7.575e-03, time/iteration:  1.335e+02 ms
2024-03-01 07:15:34,525 - INFO - [step:      27100] loss:  5.182e-03, time/iteration:  3.790e+01 ms
2024-03-01 07:15:38,162 - INFO - [step:      27200] loss:  5.797e-03, time/iteration:  

2024-03-01 07:20:02,615 - INFO - [step:      33200] loss:  1.109e-02, time/iteration:  3.673e+01 ms
2024-03-01 07:20:06,192 - INFO - [step:      33300] loss:  6.966e-03, time/iteration:  3.575e+01 ms
2024-03-01 07:20:09,543 - INFO - [step:      33400] loss:  5.399e-03, time/iteration:  3.350e+01 ms
2024-03-01 07:20:12,814 - INFO - [step:      33500] loss:  6.436e-03, time/iteration:  3.269e+01 ms
2024-03-01 07:20:16,370 - INFO - [step:      33600] loss:  1.017e-02, time/iteration:  3.554e+01 ms
2024-03-01 07:20:20,108 - INFO - [step:      33700] loss:  4.791e-03, time/iteration:  3.736e+01 ms
2024-03-01 07:20:23,668 - INFO - [step:      33800] loss:  3.542e-03, time/iteration:  3.559e+01 ms
2024-03-01 07:20:27,328 - INFO - [step:      33900] loss:  4.231e-03, time/iteration:  3.657e+01 ms
2024-03-01 07:20:41,169 - INFO - [step:      34000] record validators time:  9.603e+00s
2024-03-01 07:20:41,224 - INFO - [step:      34000] saved checkpoint to /workspace/16_2D_ChipSolidFluid_HeatTran

2024-03-01 07:25:09,945 - INFO - [step:      40000] saved checkpoint to /workspace/16_2D_ChipSolidFluid_HeatTransfer/outputs
2024-03-01 07:25:09,947 - INFO - [step:      40000] loss:  2.177e-03, time/iteration:  1.299e+02 ms
2024-03-01 07:25:09,948 - INFO - [step:      40000] reached maximum training steps, finished training!


### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

![u](./outputs/validators/validator_u.png)

![v](./outputs/validators/validator_v.png)

![p](./outputs/validators/validator_p.png)